<a href="https://colab.research.google.com/github/laibw/Handwriting-recognition/blob/main/Modeltraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from sys import getsizeof
from google.colab import files
from google.colab import drive


In [2]:
output_labels = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [3]:
def get_file_size(file_path):
  return os.path.getsize(file_path)

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/MyDrive/handwritten_data_785.csv'
df = pd.read_csv(data_path)
df.shape

(372037, 785)

In [6]:
features = df.values[:,1:]
labels = df.values[:,0]

In [7]:
features = features / 255.
features=np.ceil(features)
labels=keras.utils.to_categorical(labels)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

In [9]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(279027, 784)
(93010, 784)
(279027, 26)
(93010, 26)


In [10]:
model=keras.Sequential([
    Dense(128,activation='relu',input_shape=(784,)),
    Dense(26)
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 26)                3354      
                                                                 
Total params: 103834 (405.60 KB)
Trainable params: 103834 (405.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
del df
del features
del labels

In [14]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
8720/8720 [==============================] - 38s 4ms/step - loss: 0.3390 - accuracy: 0.9050
Epoch 2/10
8720/8720 [==============================] - 35s 4ms/step - loss: 0.1838 - accuracy: 0.9480
Epoch 3/10
8720/8720 [==============================] - 38s 4ms/step - loss: 0.1527 - accuracy: 0.9568
Epoch 4/10
8720/8720 [==============================] - 35s 4ms/step - loss: 0.1338 - accuracy: 0.9619
Epoch 5/10
8720/8720 [==============================] - 46s 5ms/step - loss: 0.1206 - accuracy: 0.9650
Epoch 6/10
8720/8720 [==============================] - 36s 4ms/step - loss: 0.1105 - accuracy: 0.9677
Epoch 7/10
8720/8720 [==============================] - 36s 4ms/step - loss: 0.1023 - accuracy: 0.9701
Epoch 8/10
8720/8720 [==============================] - 37s 4ms/step - loss: 0.0953 - accuracy: 0.9719
Epoch 9/10
8720/8720 [==============================] - 36s 4ms/step - loss: 0.0893 - accuracy: 0.9733
Epoch 10/10
8720/8720 [==============================] - 35s 4ms/step - l

In [15]:
test_loss,test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("Accuracy:",format(round(100*test_accuracy,2)))

2907/2907 - 5s - loss: 0.1415 - accuracy: 0.9643 - 5s/epoch - 2ms/step
Accuracy: 96.43


In [16]:
model_name='tf_model_AZ_handwriting.keras'
model.save(model_name)

In [17]:
print(get_file_size(model_name))

1267810


In [18]:
test_image=x_test[0:10]
test_label=y_test[0:10]
classes_y=np.argmax(test_label,axis=1)
predictions=model.predict(test_image)
classes_x=np.argmax(predictions,axis=1)
print('Actual classes: ',end="")
for q in classes_y:
  print(output_labels[q],end=" ")
print('\nPredicted classes: ',end="")
for p in classes_x:
    print(output_labels[p],end=" ")

1/1 [==============================] - 0s 88ms/step
Actual classes: J R O N Z T P O U S 
Predicted classes: J R O N Z T P O U S 

In [19]:
tflite_model_name="tf_lite_model_AZ_handwriting.tflite"
tf_lite_converter=tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_generator():
  for value in x_test:
  # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.array(value, dtype=np.float32, ndmin=2)]
tf_lite_converter.representative_dataset = representative_dataset_generator
tflite_model=tf_lite_converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [20]:
open(tflite_model_name,"wb").write(tflite_model)

106384

In [21]:
interpreter=tf.lite.Interpreter(model_path=tflite_model_name)

In [22]:
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print(input_details[0]['shape'])
print(input_details[0]['dtype'])
print(output_details[0]['shape'])
print(output_details[0]['dtype'])

[  1 784]
<class 'numpy.float32'>
[ 1 26]
<class 'numpy.float32'>


In [23]:
interpreter.resize_tensor_input(input_details[0]['index'],(93010,784))
interpreter.resize_tensor_input(output_details[0]['index'],(93010,26))
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print(input_details[0]['shape'])
print(input_details[0]['dtype'])
print(output_details[0]['shape'])
print(output_details[0]['dtype'])

[93010   784]
<class 'numpy.float32'>
[93010    26]
<class 'numpy.float32'>


In [24]:
test_image_tflite=np.array(x_test,dtype=np.float32)

In [25]:
interpreter.set_tensor(input_details[0]['index'],test_image_tflite)
interpreter.invoke()
tflite_prediction=interpreter.get_tensor(output_details[0]['index'])
prediction_class=np.argmax(tflite_prediction,axis=1)

In [26]:
actual_class=np.argmax(y_test,axis=1)
acc=accuracy_score(prediction_class,actual_class)
print(acc)

0.963950112891087


In [27]:
!apt-get -qq install xxd

In [32]:
!xxd -i /content/tf_lite_model_AZ_handwriting.tflite > tf_lite_model_AZ_handwriting.cc